# GTEx model building with CLAMP and PLIER

💡 **Environment:** `clamp-analyses`  

# CLAMP

## Load libraries

In [2]:
# Create a timestamp to track the start of the analysis
start_time <- Sys.time()
cat("GTEx CLAMP and PLIER analysis started at:", format(start_time), "\n")

GTEx CLAMP and PLIER analysis started at: 2025-11-17 13:11:15 


In [3]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER)
library(CLAMP)

source(here("config.R"))

set.seed(config$GTEx$RANDOM_SVD_SEED)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/clamp-analyses

Loading required package: RColorBrewer

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess


Loading required package: pheatmap

Loading required package: qvalue


Attaching package: ‘CLAMP’


The following object is masked from ‘package:PLIER’:

    num.pc




## Output directory

In [4]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

# Settings

In [5]:
block_size <- config$GENERAL$CHUNK_SIZE
N_CORES    <- config$GTEx$N_CORES

## Download GTEx 

In [6]:
url <- config$GTEx$URL
dest_dir <-  config$GTEx$DATASET_FOLDER
dest_gz  <- file.path(dest_dir, basename(url))

if (!file.exists(dest_gz)) {
  dir.create(dest_dir, recursive = TRUE, showWarnings = FALSE)
  download.file(url, dest_gz, mode = "wb")
  message("Downloaded to: ", dest_gz)
} else {
  message("File already exists, skipping download.")
}

File already exists, skipping download.



## Preprocess GTEx data

In [7]:
exprs_path  <- file.path(config$GTEx$DATASET_FOLDER, 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz')
output_file <- config$GTEx$DATASET_FILE

if (!file.exists(output_file)) {
  dir.create(dirname(output_file), recursive = TRUE, showWarnings = FALSE)
  exprs_data <- read.table(exprs_path, header = TRUE, sep = "\t", skip = 2, check.names = FALSE)
  saveRDS(exprs_data, config$GTEx$DATASET_FILE)
  message("File successfully written to: ", config$GTEx$DATASET_FILE)
} else {
  message("Output file already exists. Skipping.")
}

# Aggregate in-place by 'description'
gtex <- readRDS(here(config$GTEx$DATASET_FILE))
gtex <- as.data.table(gtex)
aggregated_gtex <- gtex[, lapply(.SD, sum), by = Description, .SDcols = is.numeric]

genes <- aggregated_gtex$Description
samples <- colnames(aggregated_gtex[, -1])
data_mat <- as.matrix(aggregated_gtex[, -1])

Output file already exists. Skipping.



In [8]:
fbm_files <- c(
  file.path(output_data_dir, "FBMgtex.bk"),
  file.path(output_data_dir, "FBMgtex.bk_preproc.bk"),
  file.path(output_data_dir, "FBMgtex.bk_preproc_filtered.bk"),
  file.path(output_data_dir, "FBMgtex.bk_preproc_filtered.rds")
)

for (f in fbm_files) {
  if (file.exists(f)) {
    res <- unlink(f)
    if (identical(res, 0L)) {
      message("✅ Removed existing FBM file: ", f)
    } else {
      warning("⚠️ Failed to remove: ", f)
    }
  } else {
    message("ℹ️ Not found, skipping: ", f)
  }
}

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk



✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc.bk

✅ Removed existing FBM file: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc_filtered.bk

ℹ️ Not found, skipping: /home/msubirana/Documents/pivlab/clamp-analyses/output/gtex/FBMgtex.bk_preproc_filtered.rds



In [9]:
# Create the FBM
fbm_file <- file.path(output_data_dir, "FBMgtex")
gtexFBM <- FBM(nrow = nrow(data_mat), ncol = ncol(data_mat), backingfile = fbm_file, create_bk = T)

# Populate it with data
n_blocks <- ceiling(nrow(aggregated_gtex) / block_size)

for (i in 1:n_blocks) {
  start_row <- (i-1) * block_size + 1
  end_row <- min(i * block_size, nrow(data_mat))
  
  gtexFBM[start_row:end_row, ] <- as.matrix(data_mat[start_row:end_row, ])
}

# Preprocess and z‑score FBM

prep_gtex <- preprocessCLAMPFBM(
  fbm        = gtexFBM,
  mean_cutoff= config$GTEx$GENES_MEAN_CUTOFF,
  var_cutoff = config$GTEx$GENES_VAR_CUTOFF,
  ncores=N_CORES
)

gtex_fbm_filt <- prep_gtex$fbm_filtered
gtex_rowStats <- prep_gtex$rowStats

Applying log2 transformation

No NA values found



Check overlap genes with GTEx v8 RNA-seq data and TWAS gene list

In [10]:
twas_genes <- read.csv(here('output/creating_twas_gwas_list/gene_list_union_mashr_and_elastic_net_with_phi.csv'))
head(twas_genes)

,Ensembl_ID,Gene_Symbol,Ensembl_ID_w_version,en_Adipose_Subcutaneous,en_Adipose_Visceral_Omentum,en_Adrenal_Gland,en_Artery_Aorta,en_Artery_Coronary,en_Artery_Tibial,en_Brain_Amygdala,⋯,mashr_Skin_Not_Sun_Exposed_Suprapubic,mashr_Skin_Sun_Exposed_Lower_leg,mashr_Small_Intestine_Terminal_Ileum,mashr_Spleen,mashr_Stomach,mashr_Testis,mashr_Thyroid,mashr_Uterus,mashr_Vagina,mashr_Whole_Blood
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSG00000238009,RP11-34P13.7,ENSG00000238009.6,0,0,0,0,0,0,0,⋯,0,0,0,1,0,1,0,0,0,1
2,ENSG00000228463,AP006222.2,ENSG00000228463.9,0,1,1,0,0,0,0,⋯,1,1,0,0,1,1,1,1,1,0
3,ENSG00000237094,RP4-669L17.10,ENSG00000237094.11,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,1,0,0,0
4,ENSG00000230021,RP5-857K21.4,ENSG00000230021.8,0,0,0,0,0,0,0,⋯,0,0,0,0,0,1,1,0,0,0
5,ENSG00000237491,RP11-206L10.9,ENSG00000237491.8,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1
6,ENSG00000177757,FAM87B,ENSG00000177757.2,1,1,1,1,1,1,0,⋯,1,1,1,1,1,1,1,1,1,1


In [11]:
overlap_genes <- intersect(genes[prep_gtex$kept_rows], twas_genes$Gene_Symbol)

percent_overlap <- (length(overlap_genes) / length(twas_genes$Gene_Symbol)) * 100

cat("Number of overlapping genes:", length(overlap_genes), "\n")
cat("Percentage overlap (relative to TWAS genes):", round(percent_overlap, 2), "%\n")

Number of overlapping genes: 16488 
Percentage overlap (relative to TWAS genes): 72.24 %


In [12]:
zscoreCLAMPFBM(gtex_fbm_filt, gtex_rowStats, ncores=N_CORES)
gtex_genes <- genes[prep_gtex$kept_rows]

Applying Z-score transformation



In [13]:
saveRDS(samples, file = file.path(output_data_dir, "gtex_samples.rds"))

In [14]:
saveRDS(gtex_genes, file = file.path(output_data_dir, "gtex_genes.rds"))

In [15]:
saveRDS(gtex_fbm_filt, file = file.path(output_data_dir, "gtex_fbm_filt.rds"))

In [16]:
df_gtex_fbm_filt <- as.data.frame(as.matrix(gtex_fbm_filt[]))

In [17]:
colnames(df_gtex_fbm_filt) <- samples

In [18]:
rownames(df_gtex_fbm_filt) <- gtex_genes

In [19]:
head(df_gtex_fbm_filt)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WASH7P,1.44614308,0.1466590,1.15444418,1.8411744,-0.07922225,0.6559264,1.9780371,2.5806060,1.6749832,2.04978755,⋯,-1.0148940,-0.59314323,0.65505136,-0.5693505,-0.48892918,0.2077396965,-0.8016505,-0.28439422,-1.6336384,-0.65163455
RP11-34P13.15,-0.21703290,-1.2491072,-0.71067536,-0.5458638,-0.88156172,-0.4026354,-0.5965898,0.2922380,-0.4949213,0.05590706,⋯,-0.1934056,0.33783682,1.59084836,0.4045636,0.05473854,1.0872298726,0.5077824,-0.19027836,-0.9777920,0.24859783
RP11-34P13.16,0.05000362,-1.2529810,-1.26148519,-0.7514709,-0.77242737,-0.3766090,-0.9801061,0.4096420,-0.3255995,-0.06358077,⋯,0.1692925,0.70250273,1.43810645,0.6769784,0.17310765,1.2674193527,0.8571877,-0.10447332,-1.0448862,0.46041845
RP11-34P13.18,0.80964288,-0.4648762,0.60984294,0.6852051,-1.14329360,-0.5356296,1.4716736,1.0668942,0.1565502,0.90028575,⋯,-1.3100707,-0.67305864,-0.05994370,-1.0821739,-0.70715756,0.0894613755,-1.3990557,-0.65677160,-1.7141016,-0.37942569
AP006222.2,0.60340589,-0.6590192,-0.22788724,-0.3186479,-0.75919490,-0.8022825,-0.8073561,-0.9128508,-0.3943123,-0.90819239,⋯,0.1568485,1.49184552,1.31865074,-0.1807585,0.13219755,2.1816052692,-0.3754781,-0.03196266,-0.2097524,3.06398139
MTND1P23,-0.28701222,0.4584699,-0.04655404,-0.6834497,-0.28493094,0.5112355,-0.4015529,-0.5931863,0.1299671,-0.25562386,⋯,1.0400459,-0.03813723,-0.04317319,-0.1047992,0.06352471,0.0003105984,-0.2519768,-0.01557983,0.3048702,-0.02531932


In [20]:
saveRDS(df_gtex_fbm_filt, file = file.path(output_data_dir, "df_gtex_fbm_filt.rds"))
write.csv(df_gtex_fbm_filt, file = file.path(output_data_dir, "df_gtex_fbm_filt.csv"))

## SVD computation

In [22]:
if (!file.exists(file.path(output_data_dir, "gtex_svdRes.rds"))) {

  g_fb <- nrow(gtex_fbm_filt)
  samples_fb <- ncol(gtex_fbm_filt)
  SVD_K_gtex <- min(g_fb, samples_fb) - 1

  message("Using SVD K = ", SVD_K_gtex)

  if (N_CORES > 1) {
    # disable BLAS parallelization when parallelizing
    options(bigstatsr.check.parallel.blas = FALSE)
    blas_nproc <- getOption("default.nproc.blas")
    options(default.nproc.blas = NULL)
  }

  gtex_svdRes <- big_randomSVD(
    gtex_fbm_filt,
    k = SVD_K_gtex,
    ncores = N_CORES
  )

  saveRDS(gtex_svdRes, file = file.path(output_data_dir, "gtex_svdRes.rds"))

  if (N_CORES > 1) {
    # restore previous state
    options(bigstatsr.check.parallel.blas = TRUE)
    options(default.nproc.blas = blas_nproc)
  }

} else {
  message("gtex_svdRes already exists, skipping SVD computation.")
}

gtex_svdRes already exists, skipping SVD computation.



## Estimate K for CLAMP

In [24]:
gtex_fbm_filt <- readRDS(file.path(output_data_dir, "gtex_fbm_filt.rds"))
gtex_svdRes <- readRDS(file.path(output_data_dir, "gtex_svdRes.rds"))

In [25]:
CLAMP_K_gtex <- num.pc(list(d = gtex_svdRes$d)) * 2
message("Inferred CLAMP K = ", CLAMP_K_gtex)

Inferred CLAMP K = 412



In [26]:
saveRDS(CLAMP_K_gtex, file = file.path(output_data_dir, "CLAMP_K_gtex.rds"))

## CLAMPbase

In [ ]:
gtex_baseRes <- CLAMPbase(
  Y      = gtex_fbm_filt,
  svdres = gtex_svdRes,
  trace  = TRUE,
  clamp_k = CLAMP_K_gtex
)

****

k is set to17377

L1 is set to 5.40511534491209

L2 is set to 16.2153460347363

Progress 1 / 200 | Bdiff=0.528580, minCor=0.014400

Progress 2 / 200 | Bdiff=0.108322, minCor=0.753017

Progress 3 / 200 | Bdiff=0.039332, minCor=0.940739

Progress 4 / 200 | Bdiff=0.021239, minCor=0.960508

Progress 5 / 200 | Bdiff=0.013735, minCor=0.964982

Progress 6 / 200 | Bdiff=0.009878, minCor=0.975804

Progress 7 / 200 | Bdiff=0.007549, minCor=0.977740

Progress 8 / 200 | Bdiff=0.005984, minCor=0.981555

Progress 9 / 200 | Bdiff=0.004869, minCor=0.986488

Progress 10 / 200 | Bdiff=0.004036, minCor=0.987740

Progress 11 / 200 | Bdiff=0.003398, minCor=0.989204

Progress 12 / 200 | Bdiff=0.002904, minCor=0.992270

Progress 13 / 200 | Bdiff=0.002512, minCor=0.993354

Progress 14 / 200 | Bdiff=0.002195, minCor=0.994134

Progress 15 / 200 | Bdiff=0.001938, minCor=0.994723

Progress 16 / 200 | Bdiff=0.001726, minCor=0.995027

Progress 17 / 200 | Bdiff=0.001548, minCor=0.995375

Progress 18 / 200 | Bd

In [ ]:
gtex_baseRes$Z <- data.frame(gtex_baseRes$Z)
rownames(gtex_baseRes$Z) <- gtex_genes
head(gtex_baseRes$Z)

gtex_baseRes$B <- data.frame(gtex_baseRes$B)
colnames(gtex_baseRes$B) <- samples
head(gtex_baseRes$B)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV17368,LV17369,LV17370,LV17371,LV17372,LV17373,LV17374,LV17375,LV17376,LV17377
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WASH7P,0.5704269,0.00000000,0.22462078,0.00000000,0.10910959,0.00000000,0.16922843,0.00000000,0.00000000,0.00000000,⋯,0,0.00000000,0,0,0,0,0,0,0,0.0000000
RP11-34P13.15,0.1887750,0.08500394,0.48061726,0.16617844,0.00000000,0.07042056,0.09908177,0.00000000,0.04742798,0.05550908,⋯,0,0.00000000,0,0,0,0,0,0,0,0.0000000
RP11-34P13.16,0.2304495,0.13869524,0.39541454,0.20912930,0.07341659,0.00000000,0.12522941,0.04381763,0.06524722,0.04787602,⋯,0,0.00000000,0,0,0,0,0,0,0,0.0000000
RP11-34P13.18,0.6258197,0.00000000,0.17128002,0.05285543,0.17593328,0.00000000,0.00000000,0.00000000,0.06830692,0.00000000,⋯,0,0.00000000,0,0,0,0,0,0,0,0.0000000
AP006222.2,0.2503134,0.00000000,0.26603725,0.07968998,0.15015859,0.10813655,0.00000000,0.06373656,0.07606236,0.00000000,⋯,0,0.00000000,0,0,0,0,0,0,0,0.1092002
MTND1P23,0.0000000,0.14527097,0.09360992,0.15465288,0.04618248,0.03411667,0.00000000,0.00000000,0.07518643,0.03962211,⋯,0,0.07298159,0,0,0,0,0,0,0,0.0000000


In [ ]:
saveRDS(gtex_baseRes, file = file.path(output_data_dir, "CLAMPbase.rds"))

In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPbase")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_baseRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_baseRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

## Prepare pathway priors

In [ ]:
gtex_gmtList <- list(
  KEGG = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"),
  BP = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2025"),
  GTEx_Tissues = getGMT("https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023")
)

# prefix each gene‐set name with its library to guarantee uniqueness
for(lib in names(gtex_gmtList)) {
  names(gtex_gmtList[[lib]]) <- paste0(lib, "_", names(gtex_gmtList[[lib]]))
}

gtex_pathMat <- gmtListToSparseMat(gtex_gmtList)
gtex_matched <- getMatchedPathwayMat(gtex_pathMat, gtex_genes)
gtex_chatObj <- getChat(gtex_matched)

Auto-detected name: KEGG_2021_Human

Using cached file for KEGG_2021_Human



Auto-detected name: GO_Biological_Process_2025

Using cached file for GO_Biological_Process_2025

Auto-detected name: GTEx_Tissues_V8_2023

Using cached file for GTEx_Tissues_V8_2023

There are 13687 genes in the intersection between data and prior

Removing 2030 pathways

Inverting...

done



## CLAMPfull

In [ ]:
gtex_fullRes <- CLAMPfull(
    Y = gtex_fbm_filt,
    svdres = gtex_svdRes,
    priorMat = gtex_matched,
    clamp.base.result = gtex_baseRes,
    use_cpp = TRUE,
    trace = TRUE, ,
    clamp_k = CLAMP_K_gtex
  )

In [ ]:
gtex_baseRes$Z <- data.frame(gtex_baseRes$Z)
rownames(gtex_baseRes$Z) <- gtex_genes
head(gtex_baseRes$Z)

gtex_baseRes$B <- data.frame(gtex_baseRes$B)
colnames(gtex_baseRes$B) <- samples
head(gtex_baseRes$B)

gtex_fullRes$summary <- gtex_fullRes$summary %>%
    dplyr::rename(LV = LV_index)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [ ]:
saveRDS(gtex_fullRes, file = file.path(output_data_dir, "CLAMPfull.rds"))

In [ ]:
model_dir <- file.path(output_data_dir, "CLAMPfull")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_fullRes$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_fullRes$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_fullRes$summary
write.csv(summary, file.path(model_dir, "summary.csv"))

In [ ]:
# Create a timestamp to track the end of the analysis
end_time <- Sys.time()
elapsed_time <- end_time - start_time
cat("GTEx CLAMP analysis completed at:", format(end_time), "\n")
cat("Total elapsed time:", format(elapsed_time), "\n")

GTEx PLIER2 and PLIER analysis completed at: 2025-09-13 17:49:58 
Total elapsed time: 4.797114 hours 


# PLIER

Run PLIER with the same inputs than PLIER2


In [ ]:
gtex_plier = PLIER::PLIER(
    gtex_fbm_filt[], 
    as.matrix(gtex_matched), 
    svdres = gtex_svdRes, 
    Chat = as.matrix(gtex_chatObj), 
    doCrossval = TRUE, 
    max.iter = config$GTEx$PLIER_PARAMS$MAX_ITER,
    k = CLAMP_K_gtex
  )

Removing 0 pathways with too few genes



In [ ]:
colnames(gtex_plier$Z) <- paste0('LV', seq_len(ncol(gtex_plier$Z)))

In [ ]:
head(gtex_plier$Z)
dim(gtex_plier$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0.0000000,0.02499416,0.000000000,0.00000000,0.076692203,0.00000000,0.01748135,0.00000000,0.09224386,0.0039521771,⋯,0.03193649,0.19097628,0.03509836,1.0084783,0.0000000,0.0005897013,0.00000000,0.017723577,0.00000000,0.00000000
RP11-34P13.15,0.0000000,0.03599516,0.088766434,0.66692284,0.000000000,0.40973066,0.00000000,0.00000000,0.00000000,0.0000000000,⋯,0.00000000,0.00000000,0.17049253,0.2512257,0.2381905,0.0000000000,0.07925430,0.158438333,0.06908235,0.00000000
RP11-34P13.16,0.0000000,0.04031925,0.051044799,0.68430944,0.006468101,0.44774443,0.00000000,0.00000000,0.00000000,0.0006706749,⋯,0.00000000,0.00000000,0.17280849,0.2140185,0.2540909,0.0000000000,0.09339478,0.169480128,0.07519298,0.00000000
RP11-34P13.18,0.0000000,0.00000000,0.048702159,0.00000000,0.085199947,0.24553884,0.00000000,0.00000000,0.04624262,0.0000000000,⋯,0.03087897,0.00000000,0.00916668,0.8645280,0.1606547,0.0000000000,0.03617533,0.007241217,0.00000000,0.00000000
AP006222.2,0.0000000,0.00000000,0.005665622,0.06150562,0.000000000,0.01158631,0.00000000,0.00000000,0.00000000,0.0000000000,⋯,0.00000000,0.03828548,0.01798606,0.3520241,0.2906519,0.0000000000,0.00000000,0.000000000,0.09057921,0.05701279
MTND1P23,0.2806694,0.00000000,0.000000000,0.03020411,0.055460063,0.00000000,0.06821846,0.06779852,0.00000000,0.0000000000,⋯,0.02897674,0.29230921,0.00000000,0.0000000,0.1892596,0.0866651624,0.00000000,0.196224263,0.05855393,0.20566792


[1] 21613   206

In [ ]:
gtex_plier$summary <- gtex_plier$summary %>%
    dplyr::rename(LV = `LV index`)  %>% 
    dplyr::mutate(LV = paste0('LV', LV))

In [ ]:
head(gtex_plier$summary)

,pathway,LV,AUC,p-value,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,Cells - EBV-transformed Lymphocytes Female 20-29 Up,LV1,0.9917422,7.193265e-12,2.380514e-11
2,Cells - EBV-transformed Lymphocytes Male 30-39 Up,LV1,0.9889461,2.023296e-12,7.484304e-12
3,Cells - EBV-transformed Lymphocytes Female 50-59 Up,LV1,0.9887520,2.063090e-12,7.588951e-12
4,Cells - EBV-transformed Lymphocytes Male 60-69 Up,LV1,0.9872147,2.406492e-12,8.754499e-12
5,Cells - EBV-transformed Lymphocytes Male 20-29 Up,LV1,0.9967393,9.200835e-13,3.674035e-12
6,Cells - EBV-transformed Lymphocytes Female 40-49 Up,LV1,0.9805983,4.643783e-12,1.601054e-11


In [ ]:
head(gtex_plier$B)
dim(gtex_plier$B)

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
"1,Cells - EBV-transformed Lymphocytes Male 40-49 Up",0.46617279,-0.160274908,0.73189638,0.03673337,-0.16036737,0.27011287,0.34155448,0.18709101,0.124303805,0.17580335,⋯,-0.23524018,-0.04565981,0.192526401,-0.07712182,0.18866958,0.49366655,0.083557804,0.34700683,-0.3813536716,0.23392498
"2,Colon - Transverse Male 70-79 Up",-0.05482741,0.031944407,-0.07119695,-0.03778371,0.07130929,-0.03862763,-0.10123493,-0.01607100,-0.094175281,-0.04041773,⋯,0.03768582,0.01823106,-0.086882211,0.19412630,0.28690821,-0.08885060,0.038956649,-0.06684843,0.0695568037,-0.04009313
LV 3,-0.18970150,0.084744719,-0.04421499,-0.12689688,-0.01891176,-0.03325522,-0.20798620,0.89332084,-0.003713821,0.04296174,⋯,-0.01377714,-0.08280761,0.009087268,0.09793773,0.05386236,-0.19782578,0.423212881,0.07848064,-0.0234689283,-0.05235816
"4,Whole Blood Female 50-59 Up",-0.16835215,-0.005646782,-0.14882507,-0.13028856,0.04456463,-0.14427548,-0.09085780,-0.08721934,-0.070949456,-0.13710315,⋯,0.02282449,-0.16052529,-0.080124805,-0.05021261,-0.11735479,-0.09932323,-0.088999635,-0.05632542,0.0002726316,-0.06148682
"5,Myc Targets V1",0.22784539,0.135854387,0.02489118,0.14955940,-0.35129585,0.06705728,0.04226745,0.15924572,0.305569910,0.14872014,⋯,-0.19613522,0.17144165,-0.187114670,-0.06161684,0.01157285,0.04681580,0.413042999,-0.09198360,0.2846712048,0.04444101
LV 6,-0.14773172,-0.090280367,-0.09726386,-0.17305484,-0.10486026,-0.16211997,-0.24268387,-0.18460777,-0.137521121,-0.15737880,⋯,0.05857534,0.05339165,0.067576095,0.03867730,0.09123170,0.09281796,0.009773036,0.06410904,0.0467131491,0.09168696


[1]   206 17382

In [ ]:
colnames(gtex_plier$B) <- samples

In [ ]:
saveRDS(gtex_plier, file = file.path(output_data_dir, "PLIER.rds"))

In [ ]:
model_dir <- file.path(output_data_dir, "PLIER")
dir.create(model_dir, showWarnings = FALSE, recursive = TRUE)

B <- gtex_plier$B
write.csv(B, file.path(model_dir, "B.csv"))

Z <- gtex_plier$Z
write.csv(Z, file.path(model_dir, "Z.csv"))

summary <- gtex_plier$summary
write.csv(summary, file.path(model_dir, "summary.csv"))